In [1]:
from mne.io import read_raw_persyst, read_raw_edf
from pathlib import Path

In [3]:
#edf_fpath = Path("E:/eztrack-local/data/pt_1_seeg_interictal.edf")
#raw = read_raw_edf(edf_fpath)
persyst_fpath = Path("E:/ezpersyst_test/pt_1_interictal.lay")
raw = read_raw_persyst(persyst_fpath)

Loading E:\ezpersyst_test\pt_1_interictal.lay


In [56]:
import numpy as np
from episcalp.cross_validate import ensureBalancedLabels
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

In [8]:
n_splits = 10
perc_train = 0.7
random_state = 12345

In [17]:
y = np.random.choice([0, 1], size=100, p=[.4, .6])
y

array([1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0])

In [18]:
class_1_ind = np.where(y==0)[0]
n_class_1 = len(class_1_ind)
class_2_ind = np.where(y==1)[0]
n_class_2 = len(class_2_ind)

In [14]:
n_subs = len(y)
n_train_class = round(n_subs*perc_train/2)
n_train = 2*n_train_class
n_test = n_subs-n_train
print(n_train_class, n_train, n_test)

35 70 30


In [42]:
train_ind = np.ndarray(shape=(n_splits,n_train),dtype=int)
test_ind = np.ndarray(shape=(n_splits,n_test),dtype=int)

In [43]:
for split in range(n_splits):
    rng = np.random.default_rng(split)

    # Randomly select equal number of patients from each class for training set
    perm_class1_ind = rng.permutation(n_class_1)
    perm_class2_ind = rng.permutation(n_class_2)
    train_ind1 = class_1_ind[perm_class1_ind[0:n_train_class]]
    train_ind2 = class_2_ind[perm_class2_ind[0:n_train_class]]
    _train_ind = np.concatenate([train_ind1,train_ind2])
    _train_ind = np.sort(_train_ind)
    _test_ind = np.array(range(0,n_subs))
    _test_ind = np.delete(_test_ind,_train_ind)

    train_ind[split,0:] = _train_ind
    test_ind[split,0:] = _test_ind

In [47]:
n_subs

100

In [54]:
train_ind = np.ndarray(shape=(n_splits,n_train),dtype=int)
test_ind = np.ndarray(shape=(n_splits,n_test),dtype=int)
np.random.seed(random_state)
for split in range(n_splits):
    train_ind1 = np.random.choice(class_1_ind, n_train_class, False)
    train_ind2 = np.random.choice(class_2_ind, n_train_class, False)
    train_inds_ = np.concatenate([train_ind1, train_ind2])
    train_inds = np.sort(train_inds_, kind='mergesort')
    test_inds_ = np.delete(np.arange(n_subs), train_inds)
    test_inds = np.sort(test_inds_, kind="mergesort")
    train_ind[split,:] = train_inds
    test_ind[split,:] = test_inds
train_ind, test_ind

(array([[ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 15, 17, 18,
         20, 22, 24, 25, 26, 28, 29, 31, 32, 34, 35, 36, 37, 38, 40, 41,
         43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 60, 61,
         64, 65, 66, 67, 68, 69, 71, 73, 74, 76, 78, 80, 81, 82, 83, 85,
         86, 92, 95, 96, 97, 98],
        [ 0,  1,  2,  3,  5,  6,  7,  8,  9, 12, 13, 14, 15, 16, 18, 19,
         22, 24, 25, 26, 27, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40,
         41, 44, 46, 47, 49, 50, 51, 53, 54, 56, 57, 59, 62, 63, 65, 66,
         68, 70, 71, 72, 74, 75, 76, 77, 78, 82, 83, 84, 85, 86, 88, 89,
         92, 93, 94, 95, 96, 98],
        [ 0,  1,  2,  3,  4,  5,  8,  9, 10, 11, 12, 13, 14, 15, 17, 18,
         19, 20, 21, 24, 27, 28, 29, 31, 33, 34, 35, 36, 38, 39, 43, 44,
         46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 60, 63, 64, 66,
         67, 68, 69, 70, 74, 75, 76, 77, 78, 79, 80, 82, 83, 85, 88, 89,
         92, 93, 94, 95, 98, 99],
        [ 0,  1,  2,  

In [57]:
balanced_cv = []
for train_inds, test_inds in zip(train_ind, test_ind):
    balanced_cv.append((train_inds, test_inds))
balanced_cv

[(array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 15, 17, 18, 20,
         22, 24, 25, 26, 28, 29, 31, 32, 34, 35, 36, 37, 38, 40, 41, 43, 44,
         45, 46, 48, 49, 50, 51, 52, 53, 54, 56, 57, 58, 60, 61, 64, 65, 66,
         67, 68, 69, 71, 73, 74, 76, 78, 80, 81, 82, 83, 85, 86, 92, 95, 96,
         97, 98]),
  array([ 3, 11, 16, 19, 21, 23, 27, 30, 33, 39, 42, 47, 55, 59, 62, 63, 70,
         72, 75, 77, 79, 84, 87, 88, 89, 90, 91, 93, 94, 99])),
 (array([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 12, 13, 14, 15, 16, 18, 19, 22,
         24, 25, 26, 27, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 44,
         46, 47, 49, 50, 51, 53, 54, 56, 57, 59, 62, 63, 65, 66, 68, 70, 71,
         72, 74, 75, 76, 77, 78, 82, 83, 84, 85, 86, 88, 89, 92, 93, 94, 95,
         96, 98]),
  array([ 4, 10, 11, 17, 20, 21, 23, 28, 33, 42, 43, 45, 48, 52, 55, 58, 60,
         61, 64, 67, 69, 73, 79, 80, 81, 87, 90, 91, 97, 99])),
 (array([ 0,  1,  2,  3,  4,  5,  8,  9, 10, 11, 12, 13, 14, 15,

In [59]:
clf = LogisticRegression()
X = np.random.rand(100, 5000)
scores = cross_validate(clf, X, y, cv=balanced_cv)
scores

{'fit_time': array([0.03401256, 0.03400707, 0.02100539, 0.03800845, 0.041008  ,
        0.03200698, 0.03800726, 0.03000593, 0.0150044 , 0.03900647]),
 'score_time': array([0.0010004 , 0.        , 0.        , 0.00100183, 0.00100017,
        0.00100112, 0.        , 0.0010004 , 0.0010016 , 0.00100207]),
 'test_score': array([0.56666667, 0.53333333, 0.5       , 0.43333333, 0.46666667,
        0.53333333, 0.6       , 0.56666667, 0.4       , 0.56666667])}